<a href="https://colab.research.google.com/github/leideng/AI-primer/blob/main/pytorch/block_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
times = 1
block_size = 128
batch_size = 3*times
max_num_blocks_per_req = 10*times
num_blocks = batch_size*max_num_blocks_per_req
top_k = max_num_blocks_per_req // 2
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"block_size={block_size}")
print(f"batch_size={batch_size}")
print(f"max_num_blocks_per_req={max_num_blocks_per_req}")
print(f"num_blocks={num_blocks}")
print(f"top_k={top_k}")
print(f"device={device}")

block_table = torch.randperm(num_blocks, device=device).view(batch_size, -1)
print(f"block_table=\n{block_table}")

block_size=128
batch_size=3
max_num_blocks_per_req=10
num_blocks=30
top_k=5
device=cpu
block_table=
tensor([[23, 20,  0,  3, 24,  9, 15, 28, 11,  1],
        [ 8, 27, 26, 29, 16, 14,  2, 10, 17,  7],
        [19, 25, 13, 18,  5,  6, 12, 22, 21,  4]])


In [ ]:
#torch.randint(3, 5, (3,))
seq_lens = torch.randint(low=1,high=max_num_blocks_per_req*block_size, size=(batch_size,))
print(f"seq_lens={seq_lens}")

num_valid_blocks = (seq_lens + block_size -1) // block_size
print(f"num_valid_blocks={num_valid_blocks}")

for i in range(batch_size):
  block_table[i,num_valid_blocks[i]:] = -1
print(f"block_table=\n{block_table}")

seq_lens=tensor([1172,  420,  886])
num_valid_blocks=tensor([10,  4,  7])
block_table=
tensor([[23, 20,  0,  3, 24,  9, 15, 28, 11,  1],
        [ 8, 27, 26, 29, -1, -1, -1, -1, -1, -1],
        [19, 25, 13, 18,  5,  6, 12, -1, -1, -1]])


In [ ]:
topk_block_indices = block_table[:,:top_k]

rand_num = torch.rand(batch_size, max_num_blocks_per_req)
#print(f"rand_num=\n{rand_num}")
rand_topk = torch.argsort(rand_num, dim=-1)
rand_topk = rand_topk[:,:top_k]
print(f"before sorting: rand_topk=\n{rand_topk}")
rand_topk, _ = rand_topk.sort(dim=1)
print(f"after sorting: rand_topk=\n{rand_topk}")

# Create a row index tensor that broadcasts across the batch dim
# torch.arange(batch_size) creates [0, 1, 2, ..., batch_size-1]
# .unsqueeze(1) changes it to [[0], [1], [2], ..., [batch_size-1] ]
# .expand(-1, top_k)  (say top_k=5) expands it to [[0,0,0,0,0], [1,1,1,1,1], [2,2,2,2,2], ..., [batch_size-1,batch_size-1,...,batch_size-1]]
batch_indices = torch.arange(batch_size).unsqueeze(1).expand(-1, top_k)

topk_block_indices = block_table[batch_indices, rand_topk]
print(f"topk_block_indices=\n{topk_block_indices}")

before sorting: rand_topk=
tensor([[0, 5, 6, 2, 9],
        [8, 9, 4, 2, 5],
        [6, 4, 0, 8, 7]])
after sorting: rand_topk=
tensor([[0, 2, 5, 6, 9],
        [2, 4, 5, 8, 9],
        [0, 4, 6, 7, 8]])
topk_block_indices=
tensor([[23,  0,  9, 15,  1],
        [26, -1, -1, -1, -1],
        [19,  5, 12, -1, -1]])


In [ ]:
x = torch.randint(low=1,high=100, size=(5,10))
print(f"x={x}")
idx = x >= 80
print(f"idx={idx}")
y = x[idx]
print(f"y={y}")

x=tensor([[19, 43, 91, 26,  9, 65, 85, 49,  7, 61],
        [ 7, 90, 77, 86, 85, 84, 81,  7,  3, 68],
        [ 5, 99, 90, 35, 82, 31, 84, 34, 70, 24],
        [23, 66, 16, 18, 92,  5,  5, 32, 44, 83],
        [82, 33, 90, 51, 16, 16, 68, 15, 84, 44]])
idx=tensor([[False, False,  True, False, False, False,  True, False, False, False],
        [False,  True, False,  True,  True,  True,  True, False, False, False],
        [False,  True,  True, False,  True, False,  True, False, False, False],
        [False, False, False, False,  True, False, False, False, False,  True],
        [ True, False,  True, False, False, False, False, False,  True, False]])
y=tensor([91, 85, 90, 86, 85, 84, 81, 99, 90, 82, 84, 92, 83, 82, 90, 84])
